In [28]:
import json
import pandas as pd
from iapws import IAPWS97 as w_props

In [3]:
with open('operation_points.json') as f:
    data = json.load(f)

In [7]:
data[0]['medidas']['Ms'].keys()

dict_keys(['sensor_id', 'var_id', 'description', 'unit', 'unit_SCADA', 'unit_model', 'value', 'rawData'])

In [11]:
data[0]['time']['$date']

'2009-05-15T08:51:44Z'

In [25]:
var_ids = [values['var_id'] for values in data[0]['medidas'].values()]
sensor_ids = [values['sensor_id'] for values in data[0]['medidas'].values()]
unit_scada = [values['unit_SCADA'] for values in data[0]['medidas'].values()]
unit_model = [values['unit_model'] for values in data[0]['medidas'].values()]
unit = [values['unit'] for values in data[0]['medidas'].values()]

# From var_ids extract the ones that start with "Tv_ref"
var_ids_Tv = [var_id for var_id in var_ids if var_id.startswith("Tv_ref")]
var_ids_Pv = [var_id.replace("Tv_ref", "Pv") for var_id in var_ids_Tv]

In [19]:
# Initialize an empty list to store extracted data
extracted_data = []

# Iterate through the list of dictionaries
for entry in data:
    # Extract the "time.$date" field
    time_date = entry["time"]["$date"]
    
    # Extract the "value" field for each key in the "medidas" dictionary
    medidas_values = {}
    for key, value in entry["medidas"].items():
        medidas_values[key] = value["value"]
    
    # Create a dictionary for each entry
    entry_data = {"time": time_date, **medidas_values}
    
    # Append the extracted data to the list
    extracted_data.append(entry_data)

In [31]:
# Create a DataFrame from the list of dictionaries, take variable names from var_ids
df = pd.DataFrame(extracted_data)
# Convert the "time" column to datetime
df["time"] = pd.to_datetime(df["time"])

# Rename columns with current nomenclature
column_mapping = {
    "Mprod": "Md",
    "Msw": "Mc",
    "Tprod": "Td",
    "Tcwin": "Tc_in",
    "Tcwout": "Tc_out",
    "Tvc": "Pvc",
    "Xf": "wf",
    **{old_id: new_id for new_id, old_id in zip(var_ids_Pv, var_ids_Tv)}
}
df.rename(columns=column_mapping, inplace=True)

# Change the value of "Mc" subtracting the vacuum flow (4 m³/h)
df["Mc"] = df["Mc"] - 4

# Convert "Ms" from L/s to m³/h, rho: Density, [kg/m³]
df["Ms"] = df["Ms"] * 3600 / w_props(T=df["Ts_in"].mean()+273.15, P=1.6).rho

# Convert salinity from ppm to g/kg
df["wf"] = df["wf"] / 1000

# Remove Ps
df.drop(columns=["Ps"], inplace=True)

# Filter out points that are later than July of 2012
df = df[df["time"] < "2012-08-01"]

df.head()

,time,Ms,Mf,Md,Mc,Ts_in,Ts_out,Tf,Td,Pvc,...,Pv_1,Pv_2,Pv_4,Pv_6,Pv_8,Pv_10,Pv_12,Pv_14,Tph_ref_14,wf
0,2009-05-15 08:51:44+00:00,43.989517,7.969840,2.250647,8.279807,62.513095,59.544819,57.397665,30.493447,42.446557,...,179.143093,163.495657,120.529744,99.303712,80.946400,67.065822,55.930462,44.766894,29.646541,3.30624
1,2009-05-15 09:01:44+00:00,43.990298,7.971598,2.236411,8.287988,62.290946,59.334985,57.284245,30.971941,43.394932,...,178.138166,162.795273,120.845351,100.001232,81.883529,68.147188,57.032368,45.887061,30.147909,3.30624
2,2009-05-15 09:11:44+00:00,43.989575,7.965735,2.227206,8.285425,62.581557,59.636178,57.572030,31.477519,44.579493,...,180.292725,164.509903,122.296990,101.343218,83.129892,69.374153,58.350215,47.237415,30.738334,3.30624
3,2009-05-15 09:42:52+00:00,43.986815,7.953573,2.124227,8.282549,62.680053,59.859261,57.785068,32.788266,48.240648,...,182.199579,166.838977,125.356998,104.639433,86.616237,73.060534,62.122525,51.196886,32.280036,3.30624
4,2009-05-15 09:52:52+00:00,43.982423,7.947083,2.124882,8.279844,62.885613,60.067465,58.001170,33.294130,49.769733,...,184.086935,169.043699,127.253750,106.463384,88.332202,74.731545,63.821651,52.837378,32.876461,3.30624


In [33]:
df.to_csv('data/med_model/operation_points_v2.csv', index=False)